# [LVV-T2214] - MTMount_Elevation_Changes_with_MTAOS_Aberrations

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands, plots and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

## Setting Up Test Environment

In [ ]:
test_case = "LVV-T2216"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import time

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

Use the [LVV-T2214] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(logging.ERROR)

In [ ]:
await mtcs.start_task

The cell below exposes the mount so we have a more compact code when moving the telescope carefully while M1M3 and M2 are using it to calculate their look-up tables instead of using their internal inclinometer.

In [ ]:
mtmount = mtcs.rem.mtmount

In [ ]:
script = salobj.Controller("Script", index=vandv.get_index(test_case))
await asyncio.sleep(10) 

script.start_task

In [ ]:
client = vandv.efd.create_efd_client()

---
Add a filter to all the loggers to hide the DDS read queue is filling / full.

In [ ]:
vandv.logger.add_filter_to_mtcs()

## Helper Functions

In [ ]:
async def get_data_from_efd(start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    location : str
        In which Test Stand are you running this test?
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """

    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start, 
        end=end,
    )
    
    _df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="zForce101",
        start=start, 
        end=end,
    )
        
    # M2 B1?
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
            "lutGravity0",
            "measured0"
        ],
        start=start, 
        end=end,
    )
    
    # CamHex Y position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=1,
        start=start, 
        end=end,
    )
    
    # M2Hex Y position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=2,
        start=start, 
        end=end,
    )

    # Rename columns
    _df_mount_el.rename(columns={"actualPosition": "mount_el"}, inplace=True)
    _df_m1m3_101.rename(columns={"zForce101": "m1m3_z101"}, inplace=True)
    _df_m2.rename(columns={"applied0": "m2b1_applied", "lutGravity0": "m2b1_gravLut", "measured0": "m2b1_measured"}, inplace=True)
    _df_camhex.rename(columns={"position1": "camhex_y"}, inplace=True)
    _df_m2hex.rename(columns={"position1": "m2hex_y"}, inplace=True)

    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3_101, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
         
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount
- [ ] MTRotator
- [ ] MTPtg

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Check elevation motion is smooth and no abrupt changes occur (Otherwise it faults M1M3) 

Do the following slew sequence, watch chronograph to make sure there is no abrupt change in elevation angle (otherwise it faults M1M3).

Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

Move to zenith at the end (so that we can start m1m3 with LUT in mount telemetry mode)

In [ ]:
# Move to zenith position and azimuth of 210 deg. 
warnings.simplefilter("ignore")
await vandv.mount.moveMountInElevationSteps(mtmount, 89.9, azimuth=210)

In [ ]:
# We start tracking at 85.4 deg because the track fails at 85.5 deg

target_elevations = [85.4, 84.4, 83.4, 82.4]

t_start = Time.now()
t_start.format = "isot"
log.info(f"Verifying Elevation Motion - Start time: {t_start}")

for _el in target_elevations:
    time.sleep(2.)
    az, el = await vandv.mount.moveMountInElevationSteps(mtmount, _el, azimuth=210)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end = Time.now()
t_end.format = "isot"
log.info(f"Verifying Elevation Motion - End time: {t_end}")

In [ ]:
 mtcs.rem.mtrotator.cmd_move(0)

In [ ]:
# Moving back to zenith
await vandv.mount.moveMountInElevationSteps(mtmount, 89.5, azimuth=210)

Plot elevation vs time.

In [ ]:
# Query telemetry
df = await get_data_from_efd(
    t_start, 
    t_end)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 8))

axs.plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs.set_ylabel("Mount El\n[deg]")

axs.grid(":", alpha=0.5)
axs.legend()

fig.suptitle(f"{test_exec} - Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_el_vs_time_without_aberr.png")
plt.show()

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

---
## Gather Data - Without Aberrations
Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
script.log.info(f"START - {test_case} {test_exec} Slewing without Aberrations at {Time.now()} UTC")

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]

t_start_without_aberr = Time.now()
t_start_without_aberr.format = "isot"
log.info(f"Gathering data - without Aberrations - Start time: {t_start_without_aberr}")

for _el in target_elevations:
    time.sleep(2.)
    az, el = await vandv.mount.moveMountInElevationSteps(mtmount, _el, azimuth=210)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_without_aberr = Time.now()
t_end_without_aberr.format = "isot"
log.info(f"Gathering data - without Aberrations - End time: {t_end_without_aberr}")

In [ ]:
script.log.info(f"END - {test_case} {test_exec} Slewing without Aberrations at {Time.now()}")

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
print(f"Gathering data - without Aberrations - Start time: {t_start_without_aberr}")
print(f"Gathering data - without Aberrations - End time: {t_end_without_aberr}")

In [ ]:
# Query telemetry
df = await get_data_from_efd( 
    t_start_without_aberr, 
    t_end_without_aberr)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["m1m3_z101"].dropna(), "C0o-", label="z101")
axs[1].set_ylabel("M1M3 Forces\n[N]")

#axs[2].plot(df["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].plot(df["m2b1_applied"].dropna(), "C1^-", label="applied")
axs[2].set_ylabel("M2 Forces\n[N]")

axs[3].plot(df["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_exec} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_m1m3_m2_hexs_el_vs_time_without_aberr.png")
plt.show()

---
## Gather Data - With Aberrations
Do 4 slews in sequence.
For each slew,

2 seconds before the slew starts, use MTAOS to add some aberrations to the system. The aberrations should correspond to 1um of z4 (focus), z5(astigmatism), z7 (y-coma), and z9 (trefoil), respectively track for 39s, simulating a visit.

Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
script.log.info(f"START - {test_case} {test_exec} Slewing with Aberrations (Reset every time) at {Time.now()} UTC")

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_aberr = Time.now()
t_start_with_aberr.format = "isot"
log.info(f"Gathering data - with Aberrations (reset every time) - Start time: {t_start_with_aberr}")

for _el, _ab in zip(target_elevations, aberrations):
    
    log.info(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await vandv.mount.moveMountInElevationSteps(mtmount, _el, azimuth=210)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    
t_end_with_aberr = Time.now()
t_end_with_aberr.format = "isot"
log.info(f"Gathering data - with Aberrations (reset every time) - End time: {t_end_with_aberr}")

In [ ]:
script.log.info(f"END - {test_case} {test_exec} Slewing with Aberrations (Reset every time) at {Time.now()} UTC")

### Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
print(f"Gathering data - with Aberrations (reset every time) - Start time: {t_start_with_aberr}")
print(f"Gathering data - with Aberrations (reset every time) - End time: {t_end_with_aberr}")

In [ ]:
# Query telemetry
df_aberr = await get_data_from_efd( 
    t_start_with_aberr, 
    t_end_with_aberr)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr["m1m3_z101"].dropna(), "C0o-", label="z101")
axs[1].set_ylabel("M1M3 Forces\n[N]")

#axs[2].plot(df_aberr["m2b1_measured"].dropna(), "C3-", label="Measured")
axs[2].plot(df_aberr["m2b1_applied"].dropna(), "C1^-", label="applied")
axs[2].set_ylabel("M2 Forces\n[N]")

axs[3].plot(df_aberr["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_exec} - M1M3/M2/Hexs/Elevation with Aberrations (Reset every time) vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_m1m3_m2_hexs_el_vs_time_with_aberr_reset.png")
plt.show()

## Gather Data - With Accumulated Aberrations

In [ ]:
script.log.info(f"START - {test_case} {test_exec} Slewing with Aberrations (Accumulated) at {Time.now()} UTC")

In [ ]:
# We start at 85.4 deg because the track fails at 85.5 deg
target_elevations = [85.4, 84.4, 83.4, 82.4]
aberrations = [
    0, # z4 (focus)
    1, # z5 (astigmatism)
    3, # z7 (y-coma)
    5, # z9 (trefoil)
]

await mtcs.rem.mtaos.cmd_resetCorrection.start()
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

t_start_with_aberr_acc = Time.now()
t_start_with_aberr_acc.format = "isot"
log.info(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_aberr_acc}")

for _el, _ab in zip(target_elevations, aberrations):
    
    log.info(f"\n\n\n\nCurrent elevation: {_el} - current aberration z{_ab + 4}")

    # the input for addAberration is an array with 19 elements representing 
    # the zernike coefficients starting at z4
    wavefront_errors = np.zeros(19) 
    wavefront_errors[_ab] = 1.0 # um
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)
    time.sleep(2.)
    
    az, el = await vandv.mount.moveMountInElevationSteps(mtmount, _el, azimuth=210)

    target = mtcs.radec_from_azel(az=az, el=el)
    await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

    # 39. seconds is the exposure time (2 * 15 sec) plus readout
    time.sleep(39.)
    await mtcs.stop_tracking()
    
t_end_with_aberr_acc = Time.now()
t_end_with_aberr_acc.format = "isot"
log.info(f"Gathering data - with Aberrations Accumulated - End time: {t_start_with_aberr_acc}")

In [ ]:
script.log.info(f"END - {test_case} {test_exec} Slewing with Aberrations (Accumulated) at {Time.now()} UTC")

### Plot Optics vs Time

In [ ]:
print(f"Gathering data - with Aberrations Accumulated - Start time: {t_start_with_aberr_acc}")
print(f"Gathering data - with Aberrations Accumulated - End time: {t_end_with_aberr_acc}")

In [ ]:
# Query telemetry
df_aberr_acc = await get_data_from_efd(
    t_start_with_aberr_acc, 
    t_end_with_aberr_acc)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr_acc["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr_acc["m1m3_z101"].dropna(), "C0o-", label="z101")
axs[1].set_ylabel("M1M3 Forces\n[N]")

axs[2].plot(df_aberr_acc["m2b1_applied"].dropna(), "C1^-", label="applied")
axs[2].set_ylabel("M2 Forces\n[N]")

axs[3].plot(df_aberr_acc["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr_acc["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_exec} - M1M3/M2/Hexs/Elevation with Aberrations Accum vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_exec}_m1m3_m2_hexs_el_vs_time_with_aberr_accum.png")
plt.show()

## Compare against the corrections sent by MTAOS. 

This is done in a separate notebook, LVV-T2214-plots.

## Wrap up
lower m1m3,
put hardware to OFFLINE
put all simulators to STANDBY
done.

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
await vandv.mount.moveMountInElevationSteps(mtmount, 80, azimuth=0)

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )